In [8]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from PIL import Image
import pytesseract
import cv2
import os
import imutils
import numpy as np

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [12]:
class_names = ['negative', 'positive']
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
MAX_LEN = 100

In [13]:
class SentimentClassifier(nn.Module):

    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        output = self.drop(pooled_output)
        return self.out(output)

In [14]:
model = SentimentClassifier(len(class_names))
model.load_state_dict(torch.load('best_model_state.bin'))
model = model.to(device)

In [17]:
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [22]:
file = pd.read_csv('Data Files/Test.csv')


file['Category'] = file['Category'].astype(str)
file.head()

Filename Category
0  Test1001.jpg      nan
1  Test1012.jpg      nan
2  Test1022.jpg      nan
3  Test1071.jpg      nan
4  Test1122.jpg      nan

In [26]:
j = 0
for i, row in file.iterrows():
    j+=1
    img_name = row[0]
    img_path = 'Data Files/Dataset/'+img_name
    image = cv2.imread(img_path)
    # cv2.imshow('img_name', image)
    quote = get_text(image)
    if len(quote)<5:
        file.at[i, 'Category'] = 'Random'
    else:
        review_text = quote
        encoded_review = tokenizer.encode_plus(
          review_text,
          max_length=MAX_LEN,
          add_special_tokens=True,
          return_token_type_ids=False,
          pad_to_max_length=True,
          return_attention_mask=True,
          return_tensors='pt',
          truncation=True,
        )
        
        input_ids = encoded_review['input_ids'].to(device)
        attention_mask = encoded_review['attention_mask'].to(device)

        output = model(input_ids, attention_mask)
        _, prediction = torch.max(output, dim=1)
        
        temp = class_names[prediction]
        print(temp)
        if temp=='positive':
            file.at[i, 'Category'] = 'Positive'
        else:
            file.at[i, 'Category'] = 'Negative'
    print(j, quote)
#     if j==5:
#         break
        
file.head()        

negative
1 When people ask what I see in you, I just smile and look away because Tm afraid if they knew, they'd fall in love with you too. abe i i 3 iy =a i
negative
2 a) bed mad roe ate) aod Rt a
positive
3 LOVE © LOVE ©
negative
4 pees MU Ware LCR eT em esate ine not living by society's standards, but deep down, they wish they had the courage to do the same. we Being Gay & Proud Quotes www.geckoandfly.com re * [i | xg Ff ee ees MU WATER OL em esiemelicctc aims ie not living by society's standards, but deep down, they wish they had the well reecem Cee emis] it os te Being Gay & Proud Quotes a www.geckoandfly.com —— lll
negative
5 ‘Talgher, suttear of Fale Play: How LOST Athhanes are Clalening hair (Righifel Piace in parts,” defied te nation thal sports are inhananty fmmnaphohic. Part of what is hewping pry sthiates chmetou, Zaighr said, ts tae tepor hetoresexiam ‘oi exists la Locher rocens. Negative tall about {sry people loot ths Imma. tustoad, t's the damanning tak about weenem. erg

negative
58 SR eee a Rou Oren On super_intelligent gay man who understood human problems. On the Ce ne el ed Por ee en eee ae ke ey Pe ene ee de 4 OP Re er ast ae De ee eet: Ste) See kaa e super-intelligent gay man who Pee ae es mend cross, he forgave the people who ee loving and forgiving. | don't know what makes people so cruel. Try being Pe ase you're as good as dead.” Elton John
positive
59 “People don't know if i'm gay, straight or an alien from outer space.. its funny” _ we Wy Wee ee La OEE) ke Lee an alien from outer space.. its funny” : ri ——————
negative
60 Laisa Sarva Lad Ela dived rey ned
negative
61 Ld Li Py bee Ae A ua) AND THAT Feed)
62 
negative
63 dicehc-shaamn Lemeilaenl omen ieunel sae] Pann aS eer ee ey Cet et tt TM Sal = a ebalendornigaadi [setdiadeal Leh ber iced Rieti Cs eleall Llane lene Cd Lo etlineleet beet heehteeen | ee ert ttt) ee tte la intdenedl
positive
64 “Onstage, fama devil. But I'm hardly a social reject.” ere Lae) “Onstage, | am a devil. But I'm hard

129 
130 
positive
131 stand op ond tart te fag
positive
132 CPelieve you can you're halfway T. ROOSEVELT CPehieve you can you're halfway L T. ROOSEVELT
positive
133 AGU. aGVe.
positive
134 Weare lagal" Inall fifty states" We are legal!’ In all fifty states
negative
135 ee t elneaneh reed a abinpiettapentey er el Ca altace Retieeeehe Retell pe ors
positive
136 Arn original y alveayy mirth oder than a obey,
positive
137 Arn original iy abvonugy mirth oder thane obey,
positive
138 Life's a game. Play to win. Life's a game. Play to win.
139 
negative
140 HATE It Has CAUSED A LOT OF PROBLEMS IN THIS WORLD, BUT IS HAS NOT SOLVED ONE YET. -Maya ANGELOU HATE It HAS CAUSED A LOT OF PROBLEMS IN THIS WORLD, BUT IS HAS NOT SOLVED ONE YET. -Maya ANGELou
positive
141 “Hach thine oman sands up for bervelf, without wow brag 1 ponndtely, inboerut chaivadieg bt, she stands up for all wanes” numa snares “Bach time « woman stands up for beeself, without Kerwin it pons, sw ittooart ebaiomian it,
negative


190 
191 
192 
193 
negative
194 ast Td tol [8 3b ? Ry ¢ Re
195 
196 
197 
198 
199 
200 
positive
201 apn etre meaty ha ng a me te eee Ay Bren dg De he ny hoo eer et ee ee Sw ee nm heen dy tne ny te ate
202 
negative
203 aa Had
negative
204 AMA “Ho By ¢
205 
206 
positive
207 RE-Y-to) ae mm) jaybs121|4|= ©) Added Me Everyone add my new Snapchat or comment names below. I'll be posting positive quotes every day ~ Igbtpride loveislove Igbt trans transmatter transgender transgirl transmale lesbian lesbiansex gaypride gay bi bisexualpride bisexual bipride asexual asexualpride pansexualpride pansexual protecttranskids pride swag ftm mtf JasonJ & Cit Pae ©) Added Me Everyone add my new Snapchat or comment names below. I'll be posting positive quotes every day © Igbtpride loveislove Igbt trans transmatter transgender transgirl transmale lesbian lesbiansex gaypride gay bi bisexualpride bisexual bipride asexual asexualpride pansexualpride pansexual protecttranskids pride swag ftm mtf
208 
209 T

Filename  Category
0  Test1001.jpg  Negative
1  Test1012.jpg  Negative
2  Test1022.jpg  Positive
3  Test1071.jpg  Negative
4  Test1122.jpg  Negative

In [19]:
cv2.destroyAllWindows()

In [9]:


def get_text(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray_t = cv2.threshold(gray, 0, 255,
                             cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    gray_b = cv2.medianBlur(gray, 3)

    # write the grayscale image to disk as a temporary file so we can
    # apply OCR to it
    filename = "Gray/{}.png".format(os.getpid())
    cv2.imwrite(filename, gray_t)

    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text_t = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    
    filename = "Gray/{}.png".format(os.getpid())
    cv2.imwrite(filename, gray_b)

    # load the image as a PIL/Pillow image, apply OCR, and then delete
    # the temporary file
    text_b = pytesseract.image_to_string(Image.open(filename))
    os.remove(filename)
    
    text = text_t + ' ' + text_b
    text = ' '.join(text.split())
    return text

In [27]:
print(file['Category'].value_counts())
file.head()

Random      103
Negative     79
Positive     57
Name: Category, dtype: int64


Filename  Category
0  Test1001.jpg  Negative
1  Test1012.jpg  Negative
2  Test1022.jpg  Positive
3  Test1071.jpg  Negative
4  Test1122.jpg  Negative

In [28]:
file.to_csv('Data Files/Test.csv')